In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [5]:
import csv
import random

### Trial Parsing

In [2]:
rDots_home_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/'
rDots_pcm_dir = os.path.join(rDots_home_dir, 'pcm/')
rDots_wav_dir = os.path.join(rDots_home_dir, 'wav/')
rDots_ndx_dir = os.path.join(rDots_home_dir, 'ndx/')
rDots_infos_dir = os.path.join(rDots_home_dir, 'infos/')

In [3]:
gender = 'm'
part_n = 1
prefix = "{}_part{}".format(gender, str(part_n))

### ndx

In [4]:
def rDots_parse_ndx(ndx_dir, gender, part_num):
    assert (gender in ['f', 'm'])
    assert (part_num in [1,2,3,4])
    spk_sent_ID, file_names, trial_types = [], [], []
    
    with open(ndx_dir + gender + '_part_0' + str(part_num) + '.ndx', 'r') as ndx:
        lines = ndx.readlines()
        for line in lines:
            line = line.strip()
            [spk_sent, utterance, TC, TW, IC, IW] = line.split(',')
            file_names.append(utterance+".wav")
            spk_sent_ID.append(spk_sent)

            if TC=='Y': trial_types.append(0) 
            elif TW=='Y': trial_types.append(1) 
            elif IC=='Y': trial_types.append(2) 
            elif IW=='Y': trial_types.append(3) 
    return [spk_sent_ID, file_names,  trial_types]

In [20]:
rDots_ndx_dir = os.path.join(rDots_home_dir, 'ndx/')
spk_sent_ID, file_names,trial_types = rDots_parse_ndx(rDots_ndx_dir, gender, part_n)
ndx_dict = dict(id=spk_sent_ID, file=file_names, trial_type=trial_types)

In [21]:
ndx = pd.DataFrame.from_dict(ndx_dict)
manifest_dir = "manifests/reddots/trial/{}/".format(prefix)
if not os.path.isdir(manifest_dir):
    os.makedirs(manifest_dir)
ndx.to_pickle(os.path.join(manifest_dir, "{}_ndx.pkl".format(prefix)))

### trn

In [22]:
# text-independent
if part_n == 4:
    part_n = '4_tp'

In [23]:
def rDots_parse_trn(trn_dir, gender, part_num):
    assert (gender in ['f', 'm'])
    assert (part_num in [1,2,3,'4_tp', '4_td'])
    spk_sent_ID, file_names = [], []

    with open(trn_dir + gender + '_part_0' + str(part_num) + '.trn', 'r') as trn:
        lines = trn.readlines()
        for line in lines:
            line = line.strip()
            [spk_sent, utters] = line.split(' ')
            utters = utters.split(',')
            for uttr in utters:
                spk_sent_ID.append(spk_sent)
                file_names.append(uttr+".wav")
    assert(len(spk_sent_ID) == len(file_names))
    return [spk_sent_ID, file_names]

In [24]:
rDots_trn_dir = os.path.join(rDots_home_dir, 'ndx/')
[spk_sent, file_names] = rDots_parse_trn(rDots_trn_dir, gender, part_n)
trn_dict = dict(id=spk_sent, file=file_names)

In [25]:
trn = pd.DataFrame.from_dict(trn_dict)
trn.to_pickle(os.path.join(manifest_dir,"{}_trn.pkl".format(prefix)))
trn_id = list(trn.id.unique())

## Tokenize

In [17]:
m_part1_ndx = pd.read_pickle("manifests/reddots/trial/m_part1/m_part1_ndx.pkl")
m_part1_trn = pd.read_pickle("manifests/reddots/trial/m_part1/m_part1_trn.pkl")

In [18]:
m_part1_trn_files = m_part1_trn.file.unique().tolist()
m_part1_ndx_files = m_part1_ndx.file.unique().tolist()
m_part1_files = m_part1_trn_files + m_part1_ndx_files

In [23]:
import pickle
pickle.dump(m_part1_files, open("./trials/reddots/m_part1/m_part1_files.pkl", 'wb'))

In [14]:
data_dir = "/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/token/"

In [15]:
tokens = dict()
for file in m_part1_files:
    uttr_name = file.split('/')[1][:-4]
    tokens[uttr_name] = os.listdir(os.path.join(data_dir, file[:-4]))

rows = []
for k,v in tokens.items():
    for v_i in v:
        _, spk, phrase, _ = v_i.split('_')
        rows.append([spk, phrase, k, v_i])

m_part1_token = pd.DataFrame(rows, columns=['spk', 'sent', 'utterance', 'token_file'])

# m_part1_token.to_pickle("./trials/reddots/m_part1_token_files.pkl")